In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np

In [2]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf

In [21]:
from tcn import TCN

In [4]:
lookback_window = 40

In [5]:
df = pd.read_csv("coursework_other(1).csv")

In [6]:
len(df)

9568

In [7]:
df = df.dropna()

In [8]:
len(df)

9568

In [9]:
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [10]:
y

0       463.26
1       444.37
2       488.56
3       446.48
4       473.90
         ...  
9563    460.03
9564    469.62
9565    429.57
9566    435.74
9567    453.28
Name: PE, Length: 9568, dtype: float64

In [11]:
x_scaler = MinMaxScaler()
X = x_scaler.fit_transform(X)

In [12]:
def createXY(X,y,n_splits=2):
    dataX = []
    dataY = []
    for i in range (n_splits,len(X)):
        dataX.append(X[i - n_splits:i])
        dataY.append(y[i])
    return np.array(dataX),np.array(dataY)


In [13]:
X,y = createXY(X,y,lookback_window)

In [14]:
y

array([447.14, 469.18, 482.8 , ..., 429.57, 435.74, 453.28])

In [15]:
y_scaler = MinMaxScaler()
y = y_scaler.fit_transform(y.reshape(len(y),1)).reshape(-1)

In [16]:
y

array([0.35602649, 0.64794702, 0.82834437, ..., 0.12331126, 0.20503311,
       0.43735099])

In [17]:
X.shape

(9528, 40, 4)

In [223]:
model = Sequential([
    TCN(input_shape=(lookback_window, 4),
        kernel_size=2,
        use_skip_connections=False,
        use_batch_norm=False,
        use_weight_norm=False,
        use_layer_norm=False
        ),
    Dense(1, activation='linear')
])

In [224]:
model.summary()
model.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=[tf.keras.metrics.MeanSquaredError()])

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 tcn_2 (TCN)                 (None, 64)                91712     
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 91,777
Trainable params: 91,777
Non-trainable params: 0
_________________________________________________________________


In [18]:
from sklearn.model_selection import train_test_split

In [19]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [227]:
print('Train...')
model.fit(X_train, y_train,validation_data=(X_test, y_test), epochs=100)

Train...
Epoch 1/100
239/239 [==============================] - 5s 13ms/step - loss: 2.5431 - mean_squared_error: 2.5431 - val_loss: 0.0801 - val_mean_squared_error: 0.0801
Epoch 2/100
239/239 [==============================] - 3s 12ms/step - loss: 0.0817 - mean_squared_error: 0.0817 - val_loss: 0.0568 - val_mean_squared_error: 0.0568
Epoch 3/100
239/239 [==============================] - 3s 11ms/step - loss: 0.0627 - mean_squared_error: 0.0627 - val_loss: 0.0595 - val_mean_squared_error: 0.0595
Epoch 4/100
239/239 [==============================] - 3s 12ms/step - loss: 0.0583 - mean_squared_error: 0.0583 - val_loss: 0.0651 - val_mean_squared_error: 0.0651
Epoch 5/100
239/239 [==============================] - 3s 11ms/step - loss: 0.0566 - mean_squared_error: 0.0566 - val_loss: 0.0537 - val_mean_squared_error: 0.0537
Epoch 6/100
239/239 [==============================] - 3s 11ms/step - loss: 0.0550 - mean_squared_error: 0.0550 - val_loss: 0.0576 - val_mean_squared_error: 0.0576
Epoch 7

In [229]:
model.save("tcn_b.h5")

In [23]:
model = tf.keras.models.load_model("tcn_b.h5",custom_objects={"TCN":TCN})

In [24]:
from sklearn.metrics import mean_absolute_error,mean_squared_error

In [25]:
y_pred = model.predict(X_test)

60/60 [==============================] - 4s 4ms/step


In [26]:
mean_squared_error(y_pred=y_pred,y_true=y_test)

0.022291394998818298